In [1]:
import os
import wx
import matplotlib       
matplotlib.use('WXAgg')
from matplotlib.backends.backend_wxagg import FigureCanvasWxAgg as FigureCanvas
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from wx.lib.floatcanvas import FloatCanvas

'''
This version is a combination of select image
and set rectangle to do the cropping, still
needs some improvement.
'''

class ImageSelect(wx.App):
    def __init__(self, redirect=False, filename=None):
        wx.App.__init__(self, redirect, filename)
        self.frame = wx.Frame(None, title='Image Select')
        self.panel = wx.Panel(self.frame)
        self.panel.figure = Figure()
        
        self.panel.axes = plt.Axes(self.panel.figure,[0,0,1,1])      
        self.panel.axes.set_axis_off() 
        self.panel.figure.add_axes(self.panel.axes) 
        
        self.panel.canvas = FigureCanvas(self.panel, -1, self.panel.figure)
        
        self.panel.rect = Rectangle((0,0), 1, 1, facecolor='None', edgecolor='green')
        self.panel.x0 = None
        self.panel.y0 = None
        self.panel.x1 = None
        self.panel.y1 = None
        self.panel.axes.add_patch(self.panel.rect)
        
        #self.canvas =FloatCanvas.FloatCanvas(self, BackgroundColor = "DARK SLATE BLUE")
        self.PhotoMaxSize = 224
        self.createWidgets()
        self.frame.Show()
 
    def createWidgets(self):
        instructions = 'Select an image'
        img = wx.EmptyImage(224,224)
        self.imageCtrl = wx.StaticBitmap(self.panel, wx.ID_ANY, 
                                         wx.BitmapFromImage(img))
         
        button = wx.Button(self.panel, label='crop',pos=(225, 15), size=(80, 25))    
#         button.Bind(wx.EVT_BUTTON, self.cropImage) #bind a crop action
        button.Bind(wx.EVT_PAINT, self.cropImage)
        instructLbl = wx.StaticText(self.panel, label=instructions)
        self.photoTxt = wx.TextCtrl(self.panel, size=(200,-1))
        browseBtn = wx.Button(self.panel, label='Browse', pos=(625, 15), size=(80, 25))
        browseBtn.Bind(wx.EVT_BUTTON, self.onBrowse) #bind a browse button
         
        #set a canvas the same position with the loaded image
#         self.canvas = FigureCanvas(self, -1, self.figure) #may not nede self here
        
        self.mainSizer = wx.BoxSizer(wx.VERTICAL)
        self.sizer = wx.BoxSizer(wx.HORIZONTAL)
 
        self.mainSizer.Add(wx.StaticLine(self.panel, wx.ID_ANY),
                           0, wx.ALL|wx.EXPAND, 5)
        self.mainSizer.Add(instructLbl, 0, wx.ALL, 5)
        self.mainSizer.Add(self.imageCtrl, 0, wx.ALL, 5)
        self.sizer.Add(self.photoTxt, 0, wx.ALL, 5)
        self.sizer.Add(browseBtn, 0, wx.ALL, 5)        
        self.mainSizer.Add(self.sizer, 0, wx.ALL, 5)
#         self.sizer.Add(self.panel.canvas, 3, wx.ALL)

        self.sizer.Add(self.panel.canvas, 0, wx.ALL)
#         self.SetSizer(self.sizer)
        
        self.panel.SetSizer(self.mainSizer)
        self.mainSizer.Fit(self.frame)
#         self.panel.Bind(wx.EVT_PAINT, self.on_paint)
        print('set up done')
        self.panel.Layout()
 
    def cropImage(self, event):
        
#         self.canvas = FigureCanvas(self, -1, self.figure)
        self.panel.canvas.mpl_connect('button_press_event', self._onPress)
        self.panel.canvas.mpl_connect('button_release_event', self._onRelease)
        self.panel.canvas.mpl_connect('motion_notify_event', self._onMotion)
        self.panel.pressed = False

    def onBrowse(self, event):
        """ 
        Browse for file
        """
        wildcard = "JPEG files (*.jpg)|*.jpg"
        dialog = wx.FileDialog(None, "Choose a file",
                               wildcard=wildcard,
                               style=wx.FD_OPEN)
        if dialog.ShowModal() == wx.ID_OK:
            self.photoTxt.SetValue(dialog.GetPath())
        dialog.Destroy() 
        self.onView()
 
    def onView(self):
        filepath = self.photoTxt.GetValue()
        img = wx.Image(filepath, wx.BITMAP_TYPE_ANY)
        # scale the image, preserving the aspect ratio
        W = img.GetWidth()
        H = img.GetHeight()
        if W > H:
            NewW = self.PhotoMaxSize
            NewH = self.PhotoMaxSize * H / W
        else:
            NewH = self.PhotoMaxSize
            NewW = self.PhotoMaxSize * W / H
        img = img.Scale(NewW,NewH)
 
        self.imageCtrl.SetBitmap(wx.BitmapFromImage(img))
        self.panel.Refresh()
        
    
    def _onPress(self, event):
        ''' Callback to handle the mouse being clicked and held over the canvas'''

        # Check the mouse press was actually on the canvas
        if event.xdata is not None and event.ydata is not None:

            # Upon initial press of the mouse record the origin and record the mouse as pressed
            self.panel.pressed = True
            self.panel.rect.set_linestyle('dashed')
            self.panel.x0 = event.xdata
            self.panel.y0 = event.ydata
        
        
    def _onRelease(self, event):
        '''Callback to handle the mouse being released over the canvas'''

        # Check that the mouse was actually pressed on the canvas to begin with and this isn't a rouge mouse 
        # release event that started somewhere else
        if self.panel.pressed:

            # Upon release draw the rectangle as a solid rectangle
            self.panel.pressed = False
            self.panel.rect.set_linestyle('solid')

            # Check the mouse was released on the canvas, and if it wasn't then just leave the width and 
            # height as the last values set by the motion event
            if event.xdata is not None and event.ydata is not None:
                self.panel.x1 = event.xdata
                self.panel.y1 = event.ydata

            # Set the width and height and origin of the bounding rectangle
            self.boundingRectWidth =  self.panel.x1 - self.panel.x0
            self.boundingRectHeight =  self.panel.y1 - self.panel.y0
            self.bouningRectOrigin = (self.panel.x0, self.panel.y0)

            # Draw the bounding rectangle
            self.panel.rect.set_width(self.boundingRectWidth)
            self.panel.rect.set_height(self.boundingRectHeight)
            self.panel.rect.set_xy((self.panel.x0, self.panel.y0))
            self.panel.canvas.draw()

    def _onMotion(self, event):
        '''Callback to handle the motion event created by the mouse moving over the canvas'''

        # If the mouse has been pressed draw an updated rectangle when the mouse is moved so 
        # the user can see what the current selection is
        if self.panel.pressed:

            # Check the mouse was released on the canvas, and if it wasn't then just leave the width and 
            # height as the last values set by the motion event
            if event.xdata is not None and event.ydata is not None:
                self.panel.x1 = event.xdata
                self.panel.y1 = event.ydata
            
            # Set the width and height and draw the rectangle
            self.panel.rect.set_width(self.panel.x1 - self.panel.x0)
            self.panel.rect.set_height(self.panel.y1 - self.panel.y0)
            self.panel.rect.set_xy((self.panel.x0, self.panel.y0))
            self.panel.canvas.draw()


if __name__ == '__main__':
    app = ImageSelect()
    app.MainLoop()

c:\users\cywkh\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:39: wxPyDeprecationWarning: Call to deprecated item EmptyImage. Use :class:`Image` instead.
c:\users\cywkh\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:41: wxPyDeprecationWarning: Call to deprecated item BitmapFromImage. Use :class:`wx.Bitmap` instead


set up done


c:\users\cywkh\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:110: wxPyDeprecationWarning: Call to deprecated item BitmapFromImage. Use :class:`wx.Bitmap` instead
